# Good vs Bad News

Consuming only bad news can have several detrimental effects on mental health:

1. **Increased Anxiety and Stress**: Constant exposure to negative news can lead to heightened levels of anxiety and stress. This is because the human brain is wired to respond to threats, and bad news can trigger a continuous state of alertness or fear.

2. **Depression**: Regular intake of negative news can contribute to feelings of hopelessness and despair, which are key features of depression. The perception that the world is overwhelmingly negative can lead to a pessimistic outlook on life.

3. **Desensitization**: Overexposure to bad news can cause desensitization, where individuals become numb or indifferent to distressing events. This can diminish empathy and reduce the ability to respond compassionately to real-world issues.

4. **Distorted Worldview**: Consuming only bad news can skew an individual’s perception of reality. It can create a belief that the world is more dangerous and hostile than it actually is, leading to a lack of trust in others and societal institutions.

5. **Chronic Worry**: Bad news often focuses on events beyond one's control, which can result in chronic worry. This sense of helplessness can exacerbate feelings of anxiety and stress, leading to a vicious cycle of negative thoughts.

6. **Physical Health Impacts**: Mental health and physical health are closely linked. Chronic stress and anxiety from constant exposure to bad news can lead to physical health problems such as headaches, digestive issues, and a weakened immune system.

7. **Behavioral Changes**: The psychological effects of consuming bad news can lead to behavioral changes such as social withdrawal, reduced engagement in activities, and a reluctance to take risks or try new things.

8. **Cognitive Bias**: Constant exposure to bad news can reinforce cognitive biases such as the "negativity bias," where individuals are more likely to pay attention to and remember negative information. This can further perpetuate a cycle of negative thinking.

To mitigate these effects, it’s important to balance news consumption with positive news and engage in activities that promote mental well-being, such as exercise, hobbies, and social interactions. Being mindful of media consumption and setting limits on exposure to distressing news can also help maintain a healthier mental state.

One approach could be to search the normal news stream for bad news and, based on this, search for good news on the same topics and introduce this good news into a news stream.

However, this naturally leads to the question of how to find good news and how to find bad news.

But before we start, we install some libraries necessary to get this notebook working.

In [ ]:
!pip install duckduckgo-search newspaper4k feedparser langchain-chroma langchain-openai

## How to find (bad) news?

Well, it's not that difficult. For example, you could ask a language model for search terms for bad news and you'll probably get a list like this:

In [ ]:
bad_news_keywords = ["terrorism", "hurricane", "wildfires", "deforestation", "climate change", "protests", "political instability", "corruption"]

And then the search for bad news can begin. We simply ask the Duckduckgo search engine.

In [ ]:
from duckduckgo_search import DDGS
from tqdm.notebook import tqdm
import newspaper

for result in DDGS().news(keywords=" ".join(bad_news_keywords), timelimit="m", max_results=20):
    try:
        article = newspaper.article(result['url'])
        if article.text.strip():
            print(f"# {article.title} ({article.publish_date})")
            print(article.url)
            print("")
    except Exception as ex:
        print("Could not process", result['url'])

## How to search for good news?

The results are somewhat depressing. Bad news is surprisingly easy to find. Deaths, accidents and abuse are the normal news that is reported. Ask yourself. What interests you more, news about an honest politician or a corrupt one? Normality is not news. If something is going well, it's not news. If a politician is honest, it is not news. If there hasn't been an accident at the crossroads in front of the kindergarten for ten years, it's not news either. Until a child is hit by a car. Then a crossroads with excellent accident statistics becomes news and our perception immediately changes.

That's a general problem; it's the market mechanism that journalism is subject to. This is how our attention economy works.

In other words, finding good news is far more complicated than finding bad news. Bad news is everywhere; good news is rare. But there are news outlets that deliberately focus on providing only good news in order to break through this mechanism.

If you search for "good news" on Google, you will find links like the following:

- https://thegoodnewshub.com
- https://goodnews.eu
- https://www.goodnewsnetwork.org
- https://squirrel-news.net
- ...

Again, we can search for news, but this time we limit it to positive/constructive news outlets, and since these are rare, we consider all articles in these outlets' RSS feeds.

In [ ]:
good_news_feeds = [
    "https://thegoodnewshub.com/feed/",
    "https://thegoodnewshub.com/technology/feed/",
    "https://thegoodnewshub.com/science/feed/",
    "https://thegoodnewshub.com/environment/feed/",
    "https://www.goodnewsnetwork.org/feed/",
    "https://www.goodnewsnetwork.org/category/news/world/feed/",
    "https://www.goodnewsnetwork.org/category/news/inspiring/feed/",
    "https://www.goodgoodgood.co/articles/rss.xml",
    "https://reasonstobecheerful.world/feed/",
    "https://notallnewsisbad.com/feed/",
    "https://news.janegoodall.org/feed/",
    "https://www.thegoodnewsmovement.com/feed/",
]

And with a few Python libraries, we can download these posts. Do a little research yourself. There will probably be other sources for Good News, perhaps in your native language.

In [ ]:
import feedparser
from tqdm.notebook import tqdm
import newspaper

good_news_results = { entry['link']: entry for feed in tqdm(good_news_feeds) for entry in feedparser.parse(feed).entries }

texts = []
for url, result in tqdm(good_news_results.items()):
    try:
        article = newspaper.article(url)
        if article.text.strip():
            texts.append(f"**[{article.title}]({article.url}) ({article.publish_date})**\n{article.text}")
    except Exception as ex:
        print("Could not process", url, ex)

And so we can crawl a small database of articles that deal with good news. We can just hand out five random ones.

In [ ]:
import random

for txt in random.sample(texts, 5):
    print(txt[0:500], "...")
    print()

And we can import this database into a vector store to make it semantically searchable.

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from tqdm.notebook import tqdm

embeddings = OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY", "-"))

print(f"Building semantic index for {len(texts)} articles on good news ...")
vectorstore = Chroma("good_news_store", embeddings)
for t in tqdm(texts):
    try:
        vectorstore.add_texts([t])
    except Exception as ex:
        print("Could not index", t[0:250], "...", ex)

And we can use this Vectorstore, for example, to search for good news on terms that tend to have a "bad reputation".

In [ ]:
good_news_search = vectorstore.as_retriever()
hits = good_news_search.invoke("Show me good news on wind energy")
hits

Great it works.

## A simple Bad News vs Good News search engine

And thanks to our semantically searchable database, we can now compare and contrast hits from "normal search engines" with hits from our Good News database for search terms. It's easier than you might think.

In [ ]:
from duckduckgo_search import DDGS
from ipywidgets import interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

# Interactive function to search for news
@interact_manual(
    keywords=widgets.Text(value='climate change', description='Search terms:', layout=widgets.Layout(width='75%'))
)
def search(keywords):
    display(Markdown(f"# Search engine news on '{keywords}'\n\n"))
    for result in DDGS().news(keywords=keywords, timelimit="m", max_results=5):
        display(Markdown(f"- **[{result['title']}]({result['url']})** ({result['date']})\n{result['body']}"))

    display(Markdown(f"# Good news on '{keywords}'\n\n"))
    for doc in good_news_search.invoke(keywords):
        display(Markdown(f"- {doc.page_content[0:500]} ..."))

## And now?

We have seen that we can easily search the internet for news and that this news tends to be 'bad' or focused on deficits. However, we can just as easily search for news from outlets that focus on positive and constructive news.

Here are some questions that can serve as a starting point for developing project ideas to promote positive and constructive news and make the world a better place:

1. How can we use language models to distinguish positive news sources from negative ones and suggest them to users?
2. How could a language model be used to put negative news in a positive context without distorting reality?
3. How can we develop a system that aggregates both positive and negative news and then suggests constructive solutions or courses of action?
4. How can we inspire and motivate people by regularly presenting them with positive stories and success stories?
5. How can we develop a system that analyses news in real time and encourages constructive discussion on social media?

These questions can serve as a starting point for those who want to develop innovative solutions to make the world a better place through language modelling and positive reporting.